<a href="https://colab.research.google.com/github/noemigarcia27/SImulacion-II/blob/main/Eficiencia_de_metodos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Eficiencia del Método de Monte Carlo**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random as rd
import time

In [2]:
def g(x):
  return np.sqrt(np.arctan(x))

**Monte Carlo: acierto y error**

In [33]:
def estimacion(N):
  aciertos=0
  for i in range(N):
    x = rd.random()
    y = rd.random()
    if y <= g(x):
      aciertos += 1
  I=aciertos/N
  return I

In [36]:
estimacion(1000)

0.63

Bucle para hacer el muestreo

In [34]:
def mc_acierto(N,n):
  start_time = time.perf_counter()
  lista=[]
  for i in range(N):
    E=estimacion(n)
    lista.append(E)
  end_time = time.perf_counter()
  execution_time = end_time - start_time
  return np.mean(lista), np.var(lista), execution_time

In [40]:
mc_acierto(1000,30)

(np.float64(0.6258333333333332), np.float64(0.007864861111111112))

**Monte Carlo crudo**

In [39]:
def mc_crudo(N,n):
  start_time = time.perf_counter()
  lista=[]
  for i in range(N):
    u=rd.random()
    lista.append(g(u))
  end_time = time.perf_counter()
  execution_time = end_time - start_time
  return np.mean(lista), np.var(lista)

In [38]:
mc_crudo(1000)

(np.float64(0.6325507409107539), np.float64(0.04303098438408144))